代码段10-1 一个统计单词出现频率并报告出现最频繁单词的程序。我们使用Python的dict类实现这个映射，将输入转化为小写字母并忽略所有的非字母字符

In [1]:
filename = 'README.md'
freq = {}
for piece in open(filename, encoding='utf-8').read().lower().split():
    # only consider alphabetic characters within this piece
    word = ''.join(c for c in piece if c.isalpha())
    if word:    # require at least one alphabetic character
        freq[word] = 1 + freq.get(word, 0)    # default 0
#print(freq.items())
max_word = ''
max_count = 0
for (w, c) in freq.items():    # (key, value) tuples represent (word, count)
    if c > max_count:
        max_word = w
        max_count = c
print('The most frequent word is', max_word)
print('Its number of occurrences is', max_count)

The most frequent word is 第章
Its number of occurrences is 15


代码段10-2 通过扩展MutableMapping抽象基类实现非公有类_Item，以满足各种映射应用

In [2]:
from collections import MutableMapping

<ipython-input-2-069a7babadbf>:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import MutableMapping


In [3]:
class MapBase(MutableMapping):
    """Our own abstract base class that includes a nonpublic _Item class."""
    
    # ---------- nested _Item class ----------
    class _Item:
        """Lightweight composite to store key-value pairs as map items."""
        __slots__ = '_key', '_value'
        
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key    # compare items based on their keys

        def __ne__(self, other):
            return not (self == other)        # opposite of __eq__

        def __lt__(self, other):
            return self._key < self._key      # compare items based on their keys

代码段10-3 一个用Python列表作为非排序表的map实现方法，代码段10-2给出了父类MapBase的实现

In [4]:
class UnsortedTableMap(MapBase):
    """Map implementation using an unordered list."""
    
    def __init__(self):
        """Create an empty map."""
        self._table = []    # list of _Item's
    
    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found)."""
        for item in self._table:
            if k == self._table:
                return item._value
        raise KeyError('Key Error: ' + repr(k))
    
    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if parent."""
        for item in self._table:
            if k == item._key:    # Found a match:
                item._value = v   # reassign value
                return            # and quit
        # did not find match for key
        self._table.append(self._Item(k, v))
    
    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)."""
        for j in range(len(self._table)):
            if k == self._table[j]._key:    # Found a match
                self._table.pop(j)          # remove item
                return                      # and quit
        raise KeyError('Key Error:' + repr(k))
    
    def __len__(self):
        """Return number of items in the map."""
        return len(self._table)
    
    def __iter__(self):
        """Generate iteration of the map's keys."""
        for item in self._table:
            yield item._key                 # yield the KEY

代码段10-4 一个哈希表实现的基类，基于代码段10-2中的MapBase类扩展实现的

In [7]:
class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression."""
    
    def __init__(self, cap=11, p=109345121):
        """Create an empty hash-table map."""
        self._table = cap * [None]
        self._n = 0                         # number of entyies in the map
        self._prime = p                     # prime for MAD compression
        self._scale = 1 + randrange(p-1)    # scale from 1 to p-1 for MAD
        self._shift = randrange(p)          # shift from 0 to p-1 for MAD
    
    def _hash_function(self, k):
        return (hash(k)*self._scale+self._shift) % self._prime % len(self._table)
    
    def __len__(self):
        return self._n
    
    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)           # may raise KeyError
    
    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)               # subroutine maintains self._n
        if self._n > len(self._table) // 2:         # keep load factor <= 0.5
            self._resize(2 * len(self._table) - 1)  # number 2^x - 1 is often prime
    
    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)    # may raise KeyError
        self._n -= 1
    
    def _resize(self, c):             # resize bucket array to capacity c
        old = list(self.items())      # use iteration to record existing items
        self._table = c * [None]      # then reset table to desired capacity
        self._n = 0                   # n recomputed during subsequent adds
        for (k, v) in old:
            self[k] = v               # reinsert old key-value pair

我发现我和你聊天还没有找到共同话题

代码段10-5 用分离链表实现的具体哈希map类

In [8]:
class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""
    
    def _bucket_getitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ', repr(k))    # no match found
        return bucket[k]                              # may raise KeyError
    
    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()    # bucket is new to the table
        oldsize = len(self._table[j])
        self.table[j][k] = v
        if len(self._table[j]) > oldsize:    # key was new to the table
            self._n += 1                     # increse overallmap size
    
    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))    # no match found
        del bucket[k]                                  # may raise KeyError
    
    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:    # a nonempty slot
                for key in bucket:
                    yield key

In [ ]:
def reverseString(s):
    
    for i in range(len(s)):
        